In [1]:
import torch
from pykeops.torch import LazyTensor
from gpytorch.kernels.keops import RBFKernel
from gpytorch.lazy import LazyEvaluatedKernelTensor

In [2]:
torch.set_default_dtype(torch.float64)

In [3]:
n = 1000
d = 3
sigma = 1.0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

A = torch.randn(n, d, device=device)
B = torch.randn(n, n, device=device)

PyKeops -- doesn't work

In [4]:
# Ai_lazy = LazyTensor(A[:, None, :])
# Aj_lazy = LazyTensor(A[None, :, :])
# # Compute the squared Euclidean distance matrix
# D_lazy = ((Ai_lazy - Aj_lazy) ** 2).sum(dim=2)
# D_lazy - B

GPyTorch

In [5]:
kernel = RBFKernel()
kernel.lengthscale = torch.tensor(sigma)
kernel = kernel.to(device)
# print(kernel.lengthscale)
# print(kernel.lengthscale.device)
K_lazy = kernel(A, A)

In [6]:
# Compute the actual kernel matrix via pytorch
K_true = torch.exp(-((A[:, None, :] - A[None, :, :])
                   ** 2).sum(dim=2)/(2 * sigma ** 2))

In [7]:
# torch.sum(K_lazy ** 2)

In [8]:
print(torch.allclose(K_lazy @ B, K_true @ B))
print(torch.allclose(B @ K_lazy, B @ K_true))

True
True


In [9]:
print(torch.allclose((K_lazy - B) @ B, (K_true - B) @ B))
print(torch.allclose((K_lazy - B).T @ B, (K_true - B).T @ B))
print(torch.allclose(B @ (K_lazy - B), B @ (K_true - B)))

True
True
True


In [10]:
print(torch.allclose(B[:, :100] @ (K_lazy - B)
      [:100], B[:, :100] @ (K_true - B)[:100]))

True


In [11]:
K_lazy.device

device(type='cuda', index=0)

In [12]:
K_lazy.to_dense()

tensor([[1.0000e+00, 1.4668e-01, 2.1262e-03,  ..., 3.8228e-04, 7.9677e-02,
         1.8245e-02],
        [1.4668e-01, 1.0000e+00, 9.8671e-03,  ..., 1.4180e-02, 1.7681e-01,
         3.3580e-01],
        [2.1262e-03, 9.8671e-03, 1.0000e+00,  ..., 5.3679e-01, 5.2433e-02,
         3.8000e-02],
        ...,
        [3.8228e-04, 1.4180e-02, 5.3679e-01,  ..., 1.0000e+00, 2.2496e-02,
         5.8440e-02],
        [7.9677e-02, 1.7681e-01, 5.2433e-02,  ..., 2.2496e-02, 1.0000e+00,
         5.1861e-01],
        [1.8245e-02, 3.3580e-01, 3.8000e-02,  ..., 5.8440e-02, 5.1861e-01,
         1.0000e+00]], device='cuda:0', grad_fn=<MatmulBackward>)

In [13]:
(K_lazy - B).to_dense()

tensor([[ 0.0800,  0.0181, -0.3800,  ..., -0.2621, -1.0558,  1.8005],
        [-0.9667,  0.1911,  1.9931,  ..., -1.1898,  1.2382,  0.8369],
        [-0.7898,  0.7323, -1.2224,  ...,  0.3513,  0.9323,  2.1908],
        ...,
        [-1.7796,  0.0531,  0.8696,  ..., -0.7660, -1.0400,  1.0922],
        [-0.2834, -1.0543, -1.5351,  ..., -0.1480,  0.5543,  0.3143],
        [-0.8730,  2.1096, -0.2310,  ..., -0.5507,  2.9069, -1.4745]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [14]:
type(K_lazy-B)

linear_operator.operators.sum_linear_operator.SumLinearOperator

In [15]:
from linear_operator.operators import DenseLinearOperator, CatLinearOperator

In [16]:
B_linop = DenseLinearOperator(B)
B_K_lazy_linop = CatLinearOperator(B_linop, K_lazy, dim=1)

In [17]:
B_K_lazy_linop.shape

torch.Size([1000, 2000])